In [5]:
!pip install keras==2.2.4

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple


In [1]:
from keras.layers import Dense, Input, LSTM, Embedding, BatchNormalization,Bidirectional,GlobalMaxPool1D
import os
import gc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import pandas as pd
import numpy as np
import keras
import h5py
from keras.optimizers import RMSprop


def buid_model():
    # LSTM 模型
    print('lstm model start...\n')

    # 标题输入：接收一个含有 200 个整数的序列，每个整数在 1 到 3812202 之间。
    # 标题输入：接收一个含有 200 个整数的序列，每个整数在 1 到 3812202 之间。
    main_input1 = Input(shape=(200,), name='main_input1', dtype='int32')
    with h5py.File('../../../get_w2v_feat/w2v1_pre_ad/embeddings_matrix_ad_size_32_w200_count_0.h5','r') as f:
        embeddings_matrix1 = np.array(f.get('embeddings_matrix'))
    embeddings1 = Embedding(input_dim = len(embeddings_matrix1), # 字典长度
                                output_dim = 32, # 词向量 长度（100）
                                weights=[embeddings_matrix1], # 重点：预训练的词向量系数
                                input_length=200, # 每句话的 最大长度（必须padding） 
                                trainable= True, # 是否在 训练的过程中 更新词向量
                                mask_zero = True)(main_input1)
    
    main_input2 = Input(shape=(200,), name='main_input2', dtype='int32')
    with h5py.File('../../../get_w2v_feat/w2v1_pre_advertiser/embeddings_matrix_advertiser_size_32_w200_count_5.h5','r') as f:
        embeddings_matrix2 = np.array(f.get('embeddings_matrix'))
    embeddings2 = Embedding(input_dim = len(embeddings_matrix2), # 字典长度
                                output_dim = 32, # 词向量 长度（100）
                                weights=[embeddings_matrix2], # 重点：预训练的词向量系数
                                input_length=200, # 每句话的 最大长度（必须padding） 
                                trainable= True, # 是否在 训练的过程中 更新词向量
                                mask_zero = True)(main_input2)
    
    main_input3 = Input(shape=(200,), name='main_input3', dtype='int32')
    with h5py.File('../../../get_w2v_feat/w2v1_pre_creative_id/embeddings_matrix_creative_size_32_w200_count_5.h5','r') as f:
        embeddings_matrix3 = np.array(f.get('embeddings_matrix'))
    embeddings3 = Embedding(input_dim = len(embeddings_matrix3), # 字典长度
                                output_dim = 32, # 词向量 长度（100）
                                weights=[embeddings_matrix3], # 重点：预训练的词向量系数
                                input_length=200, # 每句话的 最大长度（必须padding） 
                                trainable= True, # 是否在 训练的过程中 更新词向量
                                mask_zero = True)(main_input3)
    
    main_input4 = Input(shape=(200,),name='main_input4',dtype='int32')
    with h5py.File('../../../get_w2v_feat/w2v1_pre_industry/embeddings_matrix_industry_size_32_w200_count_5.h5','r') as f:
        embeddings_matrix4 = np.array(f.get('embeddings_matrix'))
    embeddings4 = Embedding(input_dim = len(embeddings_matrix4), # 字典长度
                                output_dim = 32, # 词向量 长度（32）
                                weights=[embeddings_matrix4], # 重点：预训练的词向量系数
                                input_length=200, # 每句话的 最大长度（必须padding） 
                                trainable=True, # 是否在 训练的过程中 更新词向量
                                mask_zero = False)(main_input4)
    main_input5 = Input(shape=(200,),name='main_input5',dtype='int32')
    with h5py.File('../../../get_w2v_feat/w2v1_pre_product_id/embeddings_matrix_product_id_size_32_w200_count_5.h5','r') as f:
        embeddings_matrix5 = np.array(f.get('embeddings_matrix'))
    embeddings5 = Embedding(input_dim = len(embeddings_matrix5), # 字典长度
                                output_dim = 32, # 词向量 长度（32）
                                weights=[embeddings_matrix5], # 重点：预训练的词向量系数
                                input_length=200, # 每句话的 最大长度（必须padding） 
                                trainable=True, # 是否在 训练的过程中 更新词向量
                                mask_zero = False)(main_input5)
    
    emb = keras.layers.concatenate([embeddings1, embeddings2, embeddings3,
                                    embeddings4, embeddings5])
    
    lstm_out = Bidirectional(LSTM(64,activation='softsign',return_sequences=False))(emb)
#     lstm_out = Bidirectional(LSTM(100,activation='softsign',return_sequences=False))(lstm_out)
    
#     lstm_out = GlobalMaxPool1D()(lstm_out)
    # 堆叠全连接
#     den_x = Dense(32, activation='relu')(lstm_out)
    
#     # 辅助输入数据
#     auxiliary_input = Input(shape=(48,), name='aux_input')
#     # 将辅助输入与lstm输出连接起来
    
#     lstm_out = Dense(30, activation='relu')(lstm_out)
#     lstm_out = BatchNormalization()(lstm_out)
    # 辅助损失函数
#     auxiliary_output = Dense(1, activation='sigmoid', name='aux_output')(lstm_out)
    # 主输出
    main_output = Dense(1, activation='sigmoid', name='main_output')(lstm_out)
    

    # 定义一个具有两个输入输出的模型
    model = keras.models.Model(inputs=[main_input1,main_input2,main_input3,
                                      main_input4,main_input5],
                               outputs=[main_output])  # 这里的输入输出顺序与fit时一致就好
    
    opt = RMSprop(lr=0.005,  clipnorm=1.0)
    model.compile(optimizer=opt,
                  loss={'main_output': 'binary_crossentropy'},
                 metrics=['accuracy'])

    print('lstm model geted...\n')
    return model

def data_load():
    print('loading data ... \n')

    with h5py.File('../../../get_w2v_feat/w2v1_pre_ad/word_train_ad_w2v_w200.h5', 'r') as f:
        data = np.array(f.get('word_data'))
        
    label = pd.read_csv('../../../train_preliminary/user.csv').sort_values(by=['user_id'])
    
    y_age = label['age'].values - 1
    y_age = keras.utils.np_utils.to_categorical(y_age, num_classes=10)
    y_gender = label['gender'].values - 1
    
    print('get data ... \n')

    return data,y_age,y_gender

def load_data2():
    
    with h5py.File('../../../get_w2v_feat/w2v1_pre_advertiser/word_train_advertiser_w2v_w200.h5', 'r') as f:
        data = np.array(f.get('word_data'))
    return data

def load_data3():
    
    with h5py.File('../../../get_w2v_feat/w2v1_pre_creative_id/word_train_creative_w2v_w200.h5', 'r') as f:
        data = np.array(f.get('word_data'))
    return data

def load_data4():
    
    print('loading data1 ... \n')
    with h5py.File('../../../get_w2v_feat/w2v1_pre_industry/word_train_industry_w2v_w200.h5', 'r') as f:
        data = np.array(f.get('word_data'))
    return data

def load_data5():
    
    print('loading data1 ... \n')
    with h5py.File('../../../get_w2v_feat/w2v1_pre_product_id/word_train_product_id_w2v_w200.h5', 'r') as f:
        data = np.array(f.get('word_data'))
    return data



def get_filename_for_saving(save_dir):
    return os.path.join(save_dir,
                        "cross_lstm32_comb_gender_w2v_200_pre_5in_{val_loss:.3f}-{val_acc:.3f}-{epoch:03d}-{loss:.3f}-{acc:.3f}.hdf5")

# model = buid_model()

data1,y_age,y_gender = data_load()
data2 = load_data2()
data3 = load_data3()
data4 = load_data4()
data5 = load_data5()

# print('lstm model fit...\n')
# checkpointer = keras.callbacks.ModelCheckpoint(
#     filepath=get_filename_for_saving(''),
#     save_best_only=False)
# stopping = keras.callbacks.EarlyStopping(patience=8)
# reduce_lr = keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=2, min_lr=0.0001)


# from sklearn.model_selection import StratifiedKFold
# kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2020)
# cvscores = []
# i = 0
# for train, test in kfold.split(data1,y_gender):
#     i = i+1
#     if i != 5:
#         print(i)
#         continue
        
#     model = buid_model()
#     print('lstm model geted...\n')
    
#     # Fit the model
#     model.fit({'main_input1': data1[train],'main_input2': data2[train],'main_input3': data3[train],
#               'main_input4': data4[train],'main_input5': data5[train]},#,'aux_input': train_x_sta},
#               {'main_output': y_gender[train]},
#               epochs=1, 
#               batch_size=256,
#               validation_data=({'main_input1': data1[test],'main_input2': data2[test],'main_input3': data3[test],
#                 'main_input4': data4[test],'main_input5': data5[test]},#,'aux_input': train_x_sta},
#                 {'main_output': y_gender[test]}),
#               callbacks=[checkpointer, reduce_lr, stopping])

Using TensorFlow backend.


loading data ... 

get data ... 

loading data1 ... 

loading data1 ... 



In [ ]:
def buid_model_age():
    # LSTM 模型
    print('lstm model start...\n')

    # 标题输入：接收一个含有 200 个整数的序列，每个整数在 1 到 3812202 之间。
    S_inputs1 = Input(shape=(200,),name='main_input1',dtype='int32')
    with h5py.File('../../../get_w2v_feat/w2v1_pre_ad/embeddings_matrix_ad_size_32_w200_count_0.h5','r') as f:
        embeddings_matrix1 = np.array(f.get('embeddings_matrix'))
    embeddings1 = Embedding(input_dim = len(embeddings_matrix1), # 字典长度
                                output_dim = 32, # 词向量 长度（32）
                                weights=[embeddings_matrix1], # 重点：预训练的词向量系数
                                input_length=200, # 每句话的 最大长度（必须padding） 
                                trainable=True, # 是否在 训练的过程中 更新词向量
                                mask_zero = True)(S_inputs1)

    S_inputs2 = Input(shape=(200,),name='main_input2',dtype='int32')
    with h5py.File('../../../get_w2v_feat/w2v1_pre_advertiser/embeddings_matrix_advertiser_size_32_w200_count_5.h5','r') as f:
        embeddings_matrix2 = np.array(f.get('embeddings_matrix'))
    embeddings2 = Embedding(input_dim = len(embeddings_matrix2), # 字典长度
                                output_dim = 32, # 词向量 长度（32）
                                weights=[embeddings_matrix2], # 重点：预训练的词向量系数
                                input_length=200, # 每句话的 最大长度（必须padding） 
                                trainable=True, # 是否在 训练的过程中 更新词向量
                                mask_zero = True)(S_inputs2)
    
    S_inputs3 = Input(shape=(200,),name='main_input3',dtype='int32')
    with h5py.File('../../../get_w2v_feat/w2v1_pre_creative_id/embeddings_matrix_creative_size_32_w200_count_5.h5','r') as f:
        embeddings_matrix3 = np.array(f.get('embeddings_matrix'))
    embeddings3 = Embedding(input_dim = len(embeddings_matrix3), # 字典长度
                                output_dim = 32, # 词向量 长度（32）
                                weights=[embeddings_matrix3], # 重点：预训练的词向量系数
                                input_length=200, # 每句话的 最大长度（必须padding） 
                                trainable=True, # 是否在 训练的过程中 更新词向量
                                mask_zero = True)(S_inputs3)
    
    emb = keras.layers.concatenate([embeddings1, embeddings2, embeddings3])
    lstm_out = Bidirectional(LSTM(64,activation='softsign',return_sequences=False))(emb)
    
    main_output = Dense(10, activation='softmax', name='main_output')(lstm_out)
    # 定义一个具有两个输入输出的模型
    model = keras.models.Model(inputs=[S_inputs1,S_inputs2,S_inputs3],#,auxiliary_input],
                               outputs=[main_output])  # 这里的输入输出顺序与fit时一致就好
    opt = RMSprop(lr=0.005,  clipnorm=1.0)
#     opt = Adam(lr=0.005)
    model.compile(optimizer=opt,
                  sample_weight_mode='None',#"temporal",
                  loss={'main_output': 'categorical_crossentropy'},
                 metrics=['accuracy'])
    print(model.summary())
    return model

def get_filename_for_saving(save_dir):
    return os.path.join(save_dir,
                        "cross_lstm32_comb_aver_age_adm_w200_{val_loss:.3f}-{val_acc:.3f}-{epoch:03d}-{loss:.3f}-{acc:.3f}.hdf5")

print('lstm model fit...\n')
checkpointer = keras.callbacks.ModelCheckpoint(
    filepath=get_filename_for_saving(''),
    save_best_only=False)
stopping = keras.callbacks.EarlyStopping(patience=8)
reduce_lr = keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=2, min_lr=0.0001)


from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2020)
cvscores = []
for train, test in kfold.split(data1,y_gender):
    
    model = buid_model_age()
    print('lstm model geted...\n')
    
    # Fit the model
    model.fit({'main_input1': data1[train],'main_input2': data2[train],'main_input3': data3[train] },#,'aux_input': train_x_sta},
              {'main_output': y_age[train]},
              epochs=1, 
              batch_size=256,
              validation_data=({'main_input1': data1[test],'main_input2': data2[test],'main_input3': data3[test]},
                {'main_output': y_age[test]}),
              callbacks=[checkpointer, reduce_lr, stopping])

lstm model fit...

lstm model start...






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input1 (InputLayer)        (None, 200)          0                                            
__________________________________________________________________________________________________
main_input2 (InputLayer)        (None, 200)          0                                            
__________________________________________________________________________________________________
main_input3 (InputLayer)        (None, 200)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 200, 32)      32319296    mai